#Ingestion del archivos movie_company csv


### Paso 1 - Leer el archivos multiples JSON  usando "DataframeReader"

In [0]:
dbutils.widgets.text("p_environment", "")
v_enviroment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run 
"../includes/common_functions"

In [0]:
%run
"../includes/configuration"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType

In [0]:
production_country_schema = StructType(fields=[
    StructField("movieId", IntegerType(), False),
    StructField("countryId", StringType(), True),
])

In [0]:
production_country_df = spark.read \
    .schema(production_country_schema) \
    .option("multiline", True) \
    .json(f"{bronze_folder_path}/{v_file_date}/production_country")


## Parte 2 : Renombrar las columnas y añadir nuevas columnas


In [0]:
from pyspark.sql.functions import col,concat,current_timestamp,lit



In [0]:
production_country_final_df = production_country_df \
                                .withColumnsRenamed({"movieId": "movie_id",
                                                     "countryId": "country_id"}) \
                                .withColumn("ingestion_date", current_timestamp()) \
                                .withColumn("enviroment", lit("Production")) \
                                .withColumn("file_date", lit(v_file_date))






## Parte 3. guardar en formato parquet en el datalake

In [0]:
#production_country_final_df.write.mode("overwrite").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.production_country")
merge_key = 'tgt.movie_id = src.movie_id AND tgt.country_id = src.country_id'
merge_delta_data(production_country_final_df,"/mnt/moviehistory7809/silver","movie_silver", "production_country", "file_date", merge_key)


In [0]:
%sql
select file_date,count(*)
from movie_silver.production_country
group by file_date


In [0]:
dbutils.notebook.exit("Success No 12")